<a href="https://colab.research.google.com/github/EricSiq/DeepLearning/blob/main/ANN_using_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Step 1: Load and preprocess the Iris dataset
def load_and_preprocess_iris_data():
    """
    Loads the Iris dataset, preprocesses features (normalization),
    and one-hot encodes target labels.
    Includes error handling for data loading.
    """
    try:
        print("Loading and preprocessing the Iris dataset...")
        iris = load_iris()
        X = iris.data
        y = iris.target.reshape(-1, 1) # Reshape targets for OneHotEncoder

        # Normalize features to a 0-1 range
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        print(f"Features scaled. Original min/max: {np.min(X):.2f}/{np.max(X):.2f}, Scaled min/max: {np.min(X_scaled):.2f}/{np.max(X_scaled):.2f}")

        # One-hot encode target labels
        encoder = OneHotEncoder(sparse_output=False)
        y_one_hot = encoder.fit_transform(y)
        print(f"Target labels one-hot encoded. Example: {y[0]} -> {y_one_hot[0]}")

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y_one_hot, test_size=0.2, random_state=42, stratify=y # stratify to maintain class distribution
        )
        print(f"Data split into training ({X_train.shape[0]} samples) and testing ({X_test.shape[0]} samples) sets.")
        return X_train, X_test, y_train, y_test, iris.target_names
    except Exception as e:
        raise RuntimeError(f"Error during data loading or preprocessing: {e}")

# Step 2: Define the ANN model architecture using TensorFlow/Keras
def define_ann_model(input_shape, num_classes):
    """
    Defines a Sequential Keras model for multi-class classification.
    Includes error handling for input parameters.
    """
    if not isinstance(input_shape, tuple) or len(input_shape) != 1 or not isinstance(input_shape[0], int) or input_shape[0] <= 0:
        raise ValueError("input_shape must be a tuple representing a positive integer (e.g., (4,)).")
    if not isinstance(num_classes, int) or num_classes <= 0:
        raise ValueError("num_classes must be a positive integer.")

    print("\nDefining the ANN model architecture...")
    model = tf.keras.Sequential([
        # Input layer: A Dense layer with 'input_shape' matching the number of features
        tf.keras.layers.Dense(units=10, activation='relu', input_shape=input_shape, name='input_hidden_layer'),
        # Hidden layer: Another Dense layer
        tf.keras.layers.Dense(units=8, activation='relu', name='hidden_layer'),
        # Output layer: Units equal to number of classes, softmax for multi-class probabilities
        tf.keras.layers.Dense(units=num_classes, activation='softmax', name='output_layer')
    ])
    print("Model architecture defined:")
    model.summary() # Print a summary of the model's layers
    return model

# Step 3: Compile the model
def compile_model(model, learning_rate=0.01):
    """
    Compiles the Keras model with an optimizer, loss function, and metrics.
    Includes error handling for learning rate.
    """
    if not isinstance(learning_rate, (int, float)) or not (0 < learning_rate <= 1):
        raise ValueError("Learning rate must be a number between 0 and 1.")

    print("\nCompiling the model...")
    # Using Adam optimizer, categorical_crossentropy for one-hot encoded labels, and accuracy metric
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    print("Model compiled successfully.")

# Step 4: Train the model
def train_model(model, X_train, y_train, epochs=100, batch_size=32):
    """
    Trains the compiled Keras model using the training data.
    Includes error handling for epochs and batch_size.
    """
    if not isinstance(epochs, int) or epochs <= 0:
        raise ValueError("Epochs must be a positive integer.")
    if not isinstance(batch_size, int) or batch_size <= 0:
        raise ValueError("Batch size must be a positive integer.")
    if not isinstance(X_train, np.ndarray) or not isinstance(y_train, np.ndarray):
        raise TypeError("Training inputs and targets must be numpy arrays.")
    if X_train.shape[0] != y_train.shape[0]:
        raise ValueError("Number of samples in training inputs and targets must match.")

    print(f"\nTraining the model for {epochs} epochs with a batch size of {batch_size}...")
    history = model.fit(X_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1) # verbose=1 shows progress bar
    print("Model training complete.")
    return history

# Step 5: Evaluate the model
def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained Keras model on the test data.
    Includes error handling for test data.
    """
    if not isinstance(X_test, np.ndarray) or not isinstance(y_test, np.ndarray):
        raise TypeError("Test inputs and targets must be numpy arrays.")
    if X_test.shape[0] != y_test.shape[0]:
        raise ValueError("Number of samples in test inputs and targets must match.")

    print("\nEvaluating the model on the test set...")
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy*100:.2f}%")
    return loss, accuracy

# Step 6: Make predictions and interpret results
def make_predictions(model, X_new, target_names):
    """
    Makes predictions on new data and converts probabilities to class labels.
    Includes error handling for input data and target names.
    """
    if not isinstance(X_new, np.ndarray):
        raise TypeError("New input data must be a numpy array.")
    if X_new.shape[1] != model.input_shape[1]:
        raise ValueError(f"New input data shape mismatch. Expected {model.input_shape[1]} features, got {X_new.shape[1]}.")
    if not isinstance(target_names, np.ndarray) or target_names.dtype.kind not in 'US':
        raise TypeError("target_names must be a numpy array of strings.")

    print("\nMaking predictions on new data...")
    # Get probability predictions from the model
    probabilities = model.predict(X_new, verbose=0)
    # Get the index of the highest probability (predicted class)
    predicted_classes_indices = np.argmax(probabilities, axis=1)

    # Map indices back to original class names
    predicted_class_names = [target_names[idx] for idx in predicted_classes_indices]

    print("Predictions:")
    for i, (input_sample, pred_prob, pred_class_name) in enumerate(zip(X_new, probabilities, predicted_class_names)):
        print(f"Sample {i+1}: Input features: {input_sample}, Predicted Probabilities: {np.round(pred_prob, 3)}, Predicted Class: {pred_class_name}")
    return predicted_classes_indices, predicted_class_names

# Main execution block
if __name__ == '__main__':
    try:
        # Load and preprocess data
        X_train, X_test, y_train, y_test, target_names = load_and_preprocess_iris_data()

        # Define model
        input_shape = (X_train.shape[1],) # (4,) for Iris features
        num_classes = y_train.shape[1]    # 3 for Iris species
        model = define_ann_model(input_shape, num_classes)

        # Compile model
        compile_model(model, learning_rate=0.001) # Using a slightly smaller learning rate for TensorFlow

        # Train model
        train_model(model, X_train, y_train, epochs=200, batch_size=16) # More epochs and smaller batch size for better convergence

        # Evaluate model
        evaluate_model(model, X_test, y_test)

        # Example new data for prediction (scaled to match training data)
        # Let's pick a few samples from the original X to demonstrate, then scale them
        sample_indices_for_prediction = [0, 50, 100, 75] # Original indices for Setosa, Versicolor, Virginica, and another Versicolor

        # Re-initialize scaler and fit on original X to ensure consistent scaling for new data
        full_data_scaler = MinMaxScaler()
        full_data_scaler.fit(load_iris().data) # Fit on the entire original dataset

        X_new_raw = load_iris().data[sample_indices_for_prediction]
        X_new_scaled = full_data_scaler.transform(X_new_raw)

        # Get actual labels for these samples for comparison
        y_new_actual_indices = load_iris().target[sample_indices_for_prediction]
        y_new_actual_names = [target_names[idx] for idx in y_new_actual_indices]
        print(f"\nActual classes for new samples: {y_new_actual_names}")

        # Make predictions
        predicted_classes_indices, predicted_class_names = make_predictions(model, X_new_scaled, target_names)

    except RuntimeError as e:
        print(f"Application Error: {e}")
    except ValueError as e:
        print(f"Configuration Error: {e}")
    except TypeError as e:
        print(f"Type Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Loading and preprocessing the Iris dataset...
Features scaled. Original min/max: 0.10/7.90, Scaled min/max: 0.00/1.00
Target labels one-hot encoded. Example: [0] -> [1. 0. 0.]
Data split into training (120 samples) and testing (30 samples) sets.

Defining the ANN model architecture...
Model architecture defined:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_hidden_layer (Dense)      │ (None, 10)             │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer (Dense)            │ (None, 8)              │            88 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 3)              │            27 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 165 (660.00 B)

 Trainable params: 165 (660.00 B)

 Non-trainable params: 0 (0.00 B)


Compiling the model...
Model compiled successfully.

Training the model for 200 epochs with a batch size of 16...
Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3553 - loss: 1.0801
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3872 - loss: 1.0559 
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3379 - loss: 1.0505 
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3663 - loss: 1.0282 
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3195 - loss: 1.0320  
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3563 - loss: 1.0082  
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3272 - loss: 1.0060 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3723 - loss: 0.9738  
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3506 - loss: 0.9777 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3198 - loss: 0.9814  
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 